In [1]:
import librosa
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import os

In [2]:
def extract_features(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    return np.mean(mfccs, axis=1)  # Average MFCCs for now

In [3]:
def load_data(data_dir):
    singer_names = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
    features, labels = [], []
    for i, singer in enumerate(singer_names):
        folder_path = os.path.join(data_dir, singer)
        print(f"Loading {singer} samples...")
        for audio_file in os.listdir(folder_path):
            if audio_file.endswith('.wav'):
                file_path = os.path.join(folder_path, audio_file)
                voice_features = extract_features(file_path)
                features.append(voice_features)
                labels.append(i)  # Use integer labels
    return np.array(features), np.array(labels), singer_names

In [ ]:
data_dir1 = '/home/nirjhar/CODE/greeneggsandham/training'
data_dir2 = '/home/nirjhar/CODE/greeneggsandham/testing-5s'

In [ ]:
print("Loading voice samples...")
print("TRAINING SAMPLES...")
X_train, y_train, singer_names_train = load_data(data_dir1)
print("TESTING SAMPLES...")
X_test, y_test, singer_names_test = load_data(data_dir2)
print(f"Total samples: {len(X_train) + len(X_test)}")
print(f"Features per sample: {X_train.shape[1]}")


print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test))}"

Loading voice samples...
TRAINING SAMPLES...
Loading arijit samples...


Loading lata samples...
Loading shreya samples...
TESTING SAMPLES...
Loading arijit samples...
Loading lata samples...
Loading shreya samples...
Total samples: 274
Features per sample: 13
Training samples: 213
Testing samples: 61


In [9]:
print("Tuning SVM hyperparameters...")
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.01, 0.1, 1],
    'kernel': ['rbf']
}
svm_model = svm.SVC(decision_function_shape='ovr')
grid_search = GridSearchCV(svm_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best CV accuracy: {grid_search.best_score_:.2f}")

Tuning SVM hyperparameters...
Best parameters: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
Best CV accuracy: 0.48


In [12]:
model = grid_search.best_estimator_
print("Training the SVM model...")
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.2f} ({accuracy*100:.1f}%)")

# Sample predictions
for i in range(min(20, len(X_test))):
    predicted = singer_names_test[y_pred[i]]
    actual = singer_names_test[y_test[i]]
    print(f"Actual: {actual}, Predicted: {predicted}")

Training the SVM model...
Test Accuracy: 0.72 (72.1%)
Actual: arijit, Predicted: shreya
Actual: arijit, Predicted: shreya
Actual: arijit, Predicted: shreya
Actual: arijit, Predicted: shreya
Actual: arijit, Predicted: shreya
Actual: arijit, Predicted: shreya
Actual: arijit, Predicted: shreya
Actual: arijit, Predicted: shreya
Actual: arijit, Predicted: shreya
Actual: arijit, Predicted: shreya
Actual: arijit, Predicted: shreya
Actual: arijit, Predicted: shreya
Actual: arijit, Predicted: shreya
Actual: arijit, Predicted: shreya
Actual: arijit, Predicted: shreya
Actual: lata, Predicted: lata
Actual: lata, Predicted: lata
Actual: lata, Predicted: lata
Actual: lata, Predicted: shreya
Actual: lata, Predicted: lata
